In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

/Users/eugen/opt/anaconda3/envs/CapstoneProject/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
